In [ ]:
#!/usr/bin/env


"""


make line datacubes and integrated maps
and compare applying continuum selfcal
NOTE: this is intended to be an interactive, iterative process
      so this is more a log that should be run by cutting and
      pasting into casa rather than as an executable script
      search "CHANGEME" for variables to be changed
10/9/15 MCA
"""


#  Setup 

IV_144 M2.0 II
16:00:00.62 -42:21:57.6


In [ ]:
field = 18                                     # CHANGEME 
fitspw    = '2,3,4,7,8,9'                      # line-free channels for fitting continuum
linespw   = '0,1,3,5,6,8'                      # line spectral windows (C18O, 13CO, CN)
robust     = 0.5                               # CHANGEME
imsize     = [640,640]
cell       = '0.03arcsec'
imagermode = 'csclean'
outframe   = 'lsrk'
veltype    = 'radio'
width      = '1.0km/s'
start      = '-3km/s'
nchan      = 16
xc         = 333                               # CHANGEME
yc         = 332                               # CHANGEME
in_a       = 80
out_a      = 120
aper       = 0.5
boxwidth = 300.
box = rg.box([xc-boxwidth,yc-boxwidth],[xc+boxwidth,yc+boxwidth])


#  Create continuum subtracted line datasets 

In [ ]:
uvcontsub(vis       = 'f'+str(field)+'.vis',                # full vis file for this field
          spw       = linespw,                              # line spw (for cont subtraction)
          fitspw    = fitspw,                               # cont spw
          combine   = 'spw',
          solint    = 'int',
          fitorder  = 1,
          want_cont = False)                                # should not be changed.


#  13CO line 

first try on 13CO line as thats going to be the brightest
os.system('rm -rf f'+str(field)+'_13co32*')


In [ ]:
clean(vis           = 'f'+str(field)+'.vis.contsub',
      imagename     = 'f'+str(field)+'_13co32',
      mode          = 'velocity',
      start         = start,
      width         = width,
      nchan         = nchan,
      outframe      = outframe,
      veltype       = veltype,
      restfreq      = '330.58797GHz',
      niter         = 500,
      threshold     = 0,
      interactive   = False,
      mask          = 'f18_cont_mask.crtf',
      imsize        = imsize,
      cell          = cell,
      weighting     ='briggs',
      robust        = robust,
      imagermode    = imagermode)


used continuum region in automated mode
coudln't see any emission in channel maps
use viewer to check channel maps and spectrum
make sure that velocity range is adequate and continuum subtraction ok


In [ ]:
imview(raster   = [{'file':'f'+str(field)+'_13co32.image'}],
        contour = [{'file':'f'+str(field)+'_cont.fits'}])


cannot see peaks
export cube to fits file    


In [ ]:
fbase = 'f'+str(field)+'_13co32'
os.system('rm -rf '+fbase+'.cube.fits')
exportfits(imagename=fbase+'.image',fitsimage=fbase+'.cube.fits')


redo moment0 maps (now excluding noisy first channel!!)
os.system('rm -rf f'+str(field)+'_13co32.mom0*')


In [ ]:
immoments(imagename  = 'f'+str(field)+'_13co32.image',            # CHANGEME (based on above analysis!!)
          outfile    = 'f'+str(field)+'_13co32.mom0',
          moments    = [0],
          includepix = [-10.0,100.0],
          chans      = ('range=[2km/s,6km/s]'))                      # CHANGEME (based on above analysis!!)


cannot see emission
export to fits file    


In [ ]:
fbase = 'f'+str(field)+'_13co32.mom0'
os.system('rm -rf '+fbase+'.fits')
exportfits(imagename=fbase,fitsimage=fbase+'.fits')


measure flux


In [ ]:
im_max = imstat(imagename = 'f'+str(field)+'_13co32.mom0')['max'][0]
im_rms = imstat(imagename = 'f'+str(field)+'_13co32.mom0',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_13co32.mom0',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = -52.84 mJy, rms = 34.04 mJy, S/N = -1.6
view continuum and gas


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_13co32.mom0'}],
       contour = [{'file':'f'+str(field)+'_cont.fits'}])


re-center image on source and use measure.py to get COG flux
os.system('rm -rf f'+str(field)+'_13co32.mom0_cropped*')


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_13co32.mom0_cropped.image',
             infile  = 'f'+str(field)+'_13co32.mom0.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_13co32.mom0_cropped.image',
           fitsimage = 'f'+str(field)+'_13co32.mom0_cropped.fits')


In [ ]:
Measuring COG for M/f18_13co32.mom0_cropped.fits
Assuming object center (300.0,300.0)
Background: 0.28 mJy/beam km/s
RMS in annulus 4.0-6.0 arcsec = 32.26 mJy/beam km/s
   i   radius    flux      err       snr
       (asec)    (mJy)     (mJy)
   0     0.10    -1.70     8.94     -0.2
   1     0.20    -8.50    25.99     -0.3
   2     0.30   -13.97    34.93     -0.4 <-- ND
   3     0.40   -23.85    54.60     -0.4
   4     0.50   -52.84    72.68     -0.7
   5     0.60   -84.09    74.31     -1.1
   6     0.70  -111.70    85.52     -1.3
   7     0.80  -110.18   108.61     -1.0
F = -1.70 mJy
E = 8.94 mJy
S = -0.19
D = 0.20 arcsec


#  C18O line  

don't bother with selfcal as it doesn't help...
os.system('rm -rf f'+str(field)+'_c18o32*')


In [ ]:
clean(vis           = 'f'+str(field)+'.vis.contsub',
      imagename     = 'f'+str(field)+'_c18o32',
      mode          = 'velocity',
      start         = start,
      width         = width,
      nchan         = nchan,
      outframe      = outframe,
      veltype       = veltype,
      restfreq      = '329.33055GHz',
      niter         = 500,
      threshold     = 0,
      interactive   = False,
      mask          = 'f18_cont_mask.crtf',
      imsize        = imsize,
      cell          = cell,
      weighting     ='briggs',
      robust        = robust,
      imagermode    = imagermode)


couldn't see line, just cleaned in automated mode
in continuum region for all channels
use viewer to check channel maps and spectrum
make sure that velocity range is adequate and continuum subtraction ok


In [ ]:
imview(raster   = [{'file':'f'+str(field)+'_c18o32.image'}],
        contour = [{'file':'f'+str(field)+'_cont.fits'}])


export cube to fits file    


In [ ]:
fbase = 'f'+str(field)+'_c18o32'
os.system('rm -rf '+fbase+'.cube.fits')
exportfits(imagename=fbase+'.image',fitsimage=fbase+'.cube.fits')


redo moment0 maps (now excluding noisy first channel!!)
os.system('rm -rf f'+str(field)+'_c18o32.mom0*')


In [ ]:
immoments(imagename  = 'f'+str(field)+'_c18o32.image',            # CHANGEME (based on above analysis!!)
          outfile    = 'f'+str(field)+'_c18o32.mom0',
          moments    = [0],
          includepix = [-10.0,100.0],
          chans      = ('range=[2km/s,6km/s]'))


can't see emission
export to fits file    


In [ ]:
fbase = 'f'+str(field)+'_c18o32.mom0'
os.system('rm -rf '+fbase+'.fits')
exportfits(imagename=fbase,fitsimage=fbase+'.fits')


measure flux


In [ ]:
im_rms = imstat(imagename = 'f'+str(field)+'_c18o32.mom0',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_c18o32.mom0',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = 42.35 mJy, rms = 36.09 mJy, S/N = 1.2
view continuum and gas


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_c18o32.mom0'}],
       contour = [{'file':'f'+str(field)+'_cont.fits'}])


re-center image on source and use measure.py to get COG flux
os.system('rm -rf f'+str(field)+'_c18o32.mom0_cropped*')


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_c18o32.mom0_cropped.image',
             infile  = 'f'+str(field)+'_c18o32.mom0.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_c18o32.mom0_cropped.image',
           fitsimage = 'f'+str(field)+'_c18o32.mom0_cropped.fits')


In [ ]:
Measuring COG for M/f18_c18o32.mom0_cropped.fits
Assuming object center (300.0,300.0)
Background: -0.03 mJy/beam km/s
RMS in annulus 4.0-6.0 arcsec = 38.12 mJy/beam km/s
   i   radius    flux      err       snr
       (asec)    (mJy)     (mJy)
   0     0.10     5.72     9.40      0.6
   1     0.20    32.87    32.51      1.0
   2     0.30    61.82    41.01      1.5 <-- ND
   3     0.40    47.66    52.03      0.9
   4     0.50    42.35    62.49      0.7
   5     0.60    91.98    91.80      1.0
   6     0.70   107.47    86.85      1.2
   7     0.80    66.08    98.03      0.7
F = 107.47 mJy
E = 86.85 mJy
S = 1.24
D = 1.40 arcsec


#  Image CN  

don't bother with selfcal as it doesn't help...
os.system('rm -rf f'+str(field)+'_cn32_b4sc*')


In [ ]:
clean(vis           = 'f'+str(field)+'.vis.contsub',
      imagename     = 'f'+str(field)+'_cn32_b4sc',
      mode          = 'velocity',
      start         = start,
      width         = width,
      nchan         = nchan,
      outframe      = outframe,
      veltype       = veltype,
      restfreq      = '340.24777GHz',
      niter         = 500,
      threshold     = 0,
      interactive   = False,
      mask          = 'f18_cont_mask.crtf',
      imsize        = imsize,
      cell          = cell,
      weighting     ='briggs',
      robust        = robust,
      imagermode    = imagermode)


couldn't see line, just cleaned in automated mode
in continuum region for all channels
use viewer to check channel maps and spectrum
make sure that velocity range is adequate and continuum subtraction ok


In [ ]:
imview(raster   = [{'file':'f'+str(field)+'_cn32_b4sc.image'}],
        contour = [{'file':'f'+str(field)+'_cont.fits'}])


can't see emission
redo moment0 maps (now excluding noisy first channel!!)
os.system('rm -rf f'+str(field)+'_cn32.mom0*')


In [ ]:
immoments(imagename  = 'f'+str(field)+'_cn32_b4sc.image',            # CHANGEME (based on above analysis!!)
          outfile    = 'f'+str(field)+'_cn32.mom0',
          moments    = [0],
          includepix = [-10.0,100.0],
          chans      = ('range=[1km/s,7km/s]'))                      # CHANGEME (based on above analysis!!)


can't see emission
export to fits file    


In [ ]:
fbase = 'f'+str(field)+'_cn32.mom0'
os.system('rm -rf '+fbase+'.fits')
exportfits(imagename=fbase,fitsimage=fbase+'.fits')


measure flux


In [ ]:
im_rms = imstat(imagename = 'f'+str(field)+'_cn32.mom0',
                region='annulus[['+str(xc)+'pix,'+str(yc)+'pix],['+str(in_a)+'pix,'+str(out_a)+'pix]]')['rms'][0]
im_flux = imstat(imagename = 'f'+str(field)+'_cn32.mom0',
                 region='circle[['+str(xc)+'pix,'+str(yc)+'pix],'+str(aper)+'arcsec]')['flux'][0]
print 'Flux = {0:.2f} mJy, rms = {1:.2f} mJy, S/N = {2:.1f}'.format(1000*im_flux, 1000*im_rms, im_flux/im_rms)


Flux = 148.93 mJy, rms = 30.16 mJy, S/N = 4.9
view continuum and gas


In [ ]:
imview(raster=[{'file':'f'+str(field)+'_cn32.mom0'}],
       contour = [{'file':'f'+str(field)+'_cont.fits'}])


re-center image on source and use measure.py to get COG flux
os.system('rm -rf f'+str(field)+'_cn32.mom0_cropped*')


In [ ]:
ia.fromimage(outfile = 'f'+str(field)+'_cn32.mom0_cropped.image',
             infile  = 'f'+str(field)+'_cn32.mom0.fits',
             region  = box )
ia.close() 
exportfits(imagename = 'f'+str(field)+'_cn32.mom0_cropped.image',
           fitsimage = 'f'+str(field)+'_cn32.mom0_cropped.fits')


COULD NOT SEE GAS, SO USED 1" APER TO MEASURE FLUX
   5    0.500     877   148.27   67.392     2.2    87.81   30.630     2.9   0.999
